In [ ]:
# pipenv install --python 3.10
# pipenv shell

# TODO:
# using pretrain embedding with transformer on smaller dataset
# using depth-wise-separable 1D convolution based model

In [36]:
# %pip install -q --upgrade keras-nlp
# %pip install -q --upgrade keras  # Upgrade to Keras 3.

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


**Setup**

In [1]:
import numpy as np 
import keras
from keras import layers
import keras_nlp
from keras_nlp import layers as nlp_layers
from pathlib import Path

2024-03-24 15:06:44.076044: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-24 15:06:44.080575: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-24 15:06:44.159000: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-24 15:06:45.866534: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


**Download the data**

In [2]:
keras.utils.get_file(
    origin="https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz",
    cache_dir="./",
    extract=True
)

imdb_dir = Path("./datasets/aclImdb")

84125825/84125825 ━━━━━━━━━━━━━━━━━━━━ 33s 0us/step


In [3]:
!tree -d datasets/aclImdb/

datasets/aclImdb/
├── test
│   ├── neg
│   └── pos
└── train
    ├── neg
    ├── pos
    └── unsup

8 directories


remove unsupervised training data, we don't need that here

In [4]:
!rm -r datasets/aclImdb/train/unsup

In [5]:
!tree -d datasets/aclImdb/

datasets/aclImdb/
├── test
│   ├── neg
│   └── pos
└── train
    ├── neg
    └── pos

7 directories


quick look at one review

In [6]:
!cat datasets/aclImdb/train/neg/21_4.txt

What was with all the Turkish actors? No offense but I thought it was all for nothing for all these actors. The film had no script to test any actors acting skill or ability. It demanded next to nothing I bought this film to see Michael Madsen. He is one of my favorite actors but this film was another failure for him. The script was so bad. Their was just nothing to sink your teeth into and all the characters were two dimensional. Madsen tried to act like a hard ass but the script and direction didn't even allow him to do enough with his character to make it more interesting or 3 dimensional.<br /><br />Even the sound effects of the gunfight at the beginning of the film sounded like the noise of paint ball guns when they are fired in a skirmish. It was really weird and they didn't sound like real guns. A video game had better sound effects than this film. There was also a really annoying bloke at the beginning of the film who was a member of the robbery gang. He had this American whini

prepare validation set

In [7]:
import os, shutil, random

validation_dir = imdb_dir / "validation"
validation_dir.mkdir()
train_dir = imdb_dir / "train"
for category in ("neg", "pos"):
    (validation_dir / category).mkdir()
    files = os.listdir(train_dir / category)
    random.Random(1234).shuffle(files)  # use seed to ensure same dataset through different runs
    num_validation_samples = int(0.2 * len(files))
    validation_files = files[-num_validation_samples:]
    for file in validation_files:
        shutil.move(train_dir / category / file,
                    validation_dir / category / file)


In [8]:
!tree -d datasets/aclImdb/

datasets/aclImdb/
├── test
│   ├── neg
│   └── pos
├── train
│   ├── neg
│   └── pos
└── validation
    ├── neg
    └── pos

10 directories


In [9]:
batch_size = 32

# 0 for negative, 1 for positive
train_dataset = keras.utils.text_dataset_from_directory(
    "datasets/aclImdb/train", batch_size=batch_size)
validation_dataset = keras.utils.text_dataset_from_directory(
    "datasets/aclImdb/validation", batch_size=batch_size)
test_dataset = keras.utils.text_dataset_from_directory(
    "datasets/aclImdb/test", batch_size=batch_size)

Found 20000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.


take a look at the batch data

In [10]:
for inputs, targets in train_dataset:
    print("inputs.shape: ", inputs.shape)
    print("inputs.dtype:", inputs.dtype)
    print("targets.shape:", targets.shape)
    print("targets.dtype:", targets.dtype)
    print("inputs[0]:", inputs[0])
    print("targets[0]:", targets[0])
    break


inputs.shape:  (32,)
inputs.dtype: <dtype: 'string'>
targets.shape: (32,)
targets.dtype: <dtype: 'int32'>
inputs[0]: tf.Tensor(b"I have to start off by apologizing because I thought the first 75-80% of this film was hilarious. It's mostly because of Brad Pitt's performance. Spot on.<br /><br />The acting by all involved was quite good but Brad stole the movie. The atmosphere was perfect in all respects. I'm not a giant Pitt fan but this has got to be one of his best roles ever.<br /><br />Brutal,Honest,Gritty. All good words to describe this movie.<br /><br />I was reading a previous review and the person said that the reasoning behind Early's violence isn't explained. It is explained but they thankfully don't have to go into graphic detail to get their point across.<br /><br />Overall I gave this a 9 because every scene bar 1 or 2 was effective. I think the humor in the first half or so is perfect for this movie. Underrated.", shape=(), dtype=string)
targets[0]: tf.Tensor(1, shape=(),

prepare a text vectorization layer

In [11]:
text_only_train_dataset = train_dataset.map(lambda x, y: x)  # do not need labels to train the text vectorization layer

max_length = 600
max_tokens = 20_000
text_vectorization = layers.TextVectorization(
    max_tokens=max_tokens,
    output_mode="int",
    output_sequence_length=max_length)

text_vectorization.adapt(text_only_train_dataset)

2024-03-24 15:12:07.206559: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


prepare integer sequence datasets 

In [12]:
int_train_dataset = train_dataset.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
int_validation_dataset = validation_dataset.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
int_test_dataset = test_dataset.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)

use keras built-in transformer encoder for text classification

In [13]:
transformer_encoder = nlp_layers.TransformerEncoder(intermediate_dim=32, 
                                                    num_heads=3)

use the transformer encoder for text classification

In [15]:
vocabulary_size = 20_000
sequence_length = 600
embed_dimension = 256

num_heads = 2
dense_layer_dimension = 32

inputs = keras.Input(shape=(None,), dtype="int64")

x = nlp_layers.TokenAndPositionEmbedding(vocabulary_size=vocabulary_size,
                                         sequence_length=sequence_length,
                                         embedding_dim=embed_dimension
                                         )(inputs)
x = nlp_layers.TransformerEncoder(intermediate_dim=dense_layer_dimension,
                                  num_heads=num_heads
                                  )(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
model.summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, None)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ token_and_position_embedding_1  │ (None, None, 256)      │     5,273,600 │
│ (TokenAndPositionEmbedding)     │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_encoder_2           │ (None, None, 256)      │       280,864 │
│ (TransformerEncoder)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling1d_1          │ (None, 256)            │             0 │
│ (GlobalMaxPooling1D)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,554,721 (21.19 MB)

 Trainable params: 5,554,721 (21.19 MB)

 Non-trainable params: 0 (0.00 B)

train the transformer encoder model

In [ ]:
callbacks = [
    keras.callbacks.ModelCheckpoint(
        "trained_models/transformer_encoder.keras",
        save_best_only=True)]

model.fit(int_train_dataset,
          validation_data=int_validation_dataset,
          epochs=3, # 20
          callbacks=callbacks)


evaluate the transformer encoder model

In [ ]:
model = keras.models.load_model(
    "trained_models/transformer_encoder.keras")

print(f"Test accuracy: {model.evaluate(int_test_dataset)[1]:.3f}")